In [5]:
library(data.table)
library(tictoc)

tic("Esercizio Totale") 

cohortA_samples <- fread("cohortA_samples.csv")
cohortB_samples <- fread("cohortB_samples.csv")
bulk_counts_long <- fread("bulk_counts_long.csv")

setnames(bulk_counts_long, old = "gene", new = "gene_id")

#TASK1
tic("Task1")

combined_samples <- rbindlist(list(cohortA_samples, cohortB_samples), 
                              use.names = TRUE, 
                              fill = TRUE)

cat("Verifica dell'allineamento delle colonne:\n")
print(names(combined_samples)) # Stampa tutti i nomi di colonna
cat("Dimensione finale (righe, colonne):", dim(combined_samples), "\n\n")

toc() 


#TASK2
tic("Task2")

setorder(combined_samples, cohort, condition, sample_id)

cat("tabella ordinata \n")
print(head(combined_samples))
cat("\n")

toc()

#TASK3
tic("Task3")

gene_variances <- bulk_counts_long[, .(variance = var(count)), by = gene_id]

top_100_genes <- gene_variances[order(-variance)][1:100, gene_id]

bulk_counts_filtered <- bulk_counts_long[gene_id %in% top_100_genes]

combined_data <- combined_samples[bulk_counts_filtered, 
                                  on = "sample_id", 
                                  nomatch = 0] # nomatch=0 assicura un inner join

mean_counts_by_group <- combined_data[, 
                                      .(mean_count = mean(count)), 
                                      by = .(cohort, condition, gene_id)]

cat("Media dei conteggi Top 100 Geni:\n")
print(head(mean_counts_by_group, 10))

toc() 

#fine timer generale
toc() 


Attaching package: ‘data.table’


The following object is masked from ‘package:tictoc’:

    shift




Verifica dell'allineamento delle colonne:
[1] "sample_id"  "condition"  "batch"      "patient_id" "timepoint" 
[6] "cohort"    
Dimensione finale (righe, colonne): 24 6 

Task1: 0.029 sec elapsed
tabella ordinata 
   sample_id condition  batch patient_id timepoint cohort
      <char>    <char> <char>     <char>    <char> <char>
1:       S19   control batch2       P013        T0      A
2:       S20   control batch3       P017        T1      A
3:       S21   control batch2       P008        T0      A
4:       S03   treated batch2       P017        T1      A
5:       S04   treated batch3       P044        T0      A
6:       S05   treated batch3       P020        T1      A

Task2: 0.052 sec elapsed
Media dei conteggi Top 100 Geni:
    cohort condition   gene_id mean_count
    <char>    <char>    <char>      <num>
 1:      B   treated GENE_0232   78.00000
 2:      B   treated GENE_0245   29.50000
 3:      B   treated GENE_0130   46.00000
 4:      B   treated GENE_0241   41.16667
 5:      B 

In [6]:
library(tictoc)

# Inizio timer generale
tic("Esercizio Totale_dataframe")

# Caricamento dati (Sostituzione di fread con read.csv)
cohortA_samples <- read.csv("cohortA_samples.csv", stringsAsFactors = FALSE)
cohortB_samples <- read.csv("cohortB_samples.csv", stringsAsFactors = FALSE)
bulk_counts_long <- read.csv("bulk_counts_long.csv", stringsAsFactors = FALSE)

# Rinominazione colonna (Sostituzione di setnames)
names(bulk_counts_long)[names(bulk_counts_long) == "gene"] <- "gene_id"

# ----------------------------------------------------

## 🏃 TASK 1: Unione dei data.frame
tic("Task1_dataframe")

# Sostituzione di rbindlist con rbind (per data.frame)
combined_samples <- rbind(cohortA_samples, cohortB_samples)

cat("Verifica dell'allineamento delle colonne:\n")
print(names(combined_samples)) 
cat("Dimensione finale (righe, colonne):", dim(combined_samples), "\n\n")

toc() 


#TASK 2: Ordinamento
tic("Task2_dataframe")

# Ordinamento del data.frame base in base a più colonne (Sostituzione di setorder)
combined_samples <- combined_samples[
    order(combined_samples$cohort, combined_samples$condition, combined_samples$sample_id), 
]

cat("Tabella ordinata (prime 6 righe)\n")
print(head(combined_samples))
cat("\n")

toc()


#TASK 3: Varianza, Filtraggio e Media
tic("Task3_dataframe")

# 1. Calcolo della varianza per gene (Sostituzione di DT[, .(var()), by = gene_id])
gene_variances <- aggregate(
    bulk_counts_long$count, 
    by = list(gene_id = bulk_counts_long$gene_id), 
    FUN = var
)
names(gene_variances)[names(gene_variances) == "x"] <- "variance"


# 2. Selezione dei Top 100 geni per varianza
# Ordinamento e selezione
gene_variances_ordered <- gene_variances[order(gene_variances$variance, decreasing = TRUE), ]
top_100_genes <- head(gene_variances_ordered, 100)$gene_id


# 3. Filtraggio dei conteggi (Sostituzione di DT[col %in% values])
bulk_counts_filtered <- bulk_counts_long[bulk_counts_long$gene_id %in% top_100_genes, ]


# 4. Unione dei dati (Sostituzione di DT[DT2, on=..., nomatch=0] con merge)
# all = FALSE assicura un inner join (equivalente a nomatch=0)
combined_data <- merge(
    x = bulk_counts_filtered, 
    y = combined_samples, 
    by = "sample_id", 
    all = FALSE 
)


# 5. Calcolo della media dei conteggi per gruppo (Sostituzione di DT[, .(mean()), by = ...])
mean_counts_by_group <- aggregate(
    combined_data$count, 
    by = list(
        cohort = combined_data$cohort, 
        condition = combined_data$condition, 
        gene_id = combined_data$gene_id
    ), 
    FUN = mean
)
names(mean_counts_by_group)[names(mean_counts_by_group) == "x"] <- "mean_count"

cat("Media dei conteggi Top 100 Geni (prime 10 righe):\n")
print(head(mean_counts_by_group, 10))

toc()

# Fine timer generale
toc()

Verifica dell'allineamento delle colonne:
[1] "sample_id"  "condition"  "batch"      "patient_id" "timepoint" 
[6] "cohort"    
Dimensione finale (righe, colonne): 24 6 

Task1_dataframe: 0.02 sec elapsed
Tabella ordinata (prime 6 righe)
   sample_id condition  batch patient_id timepoint cohort
2        S19   control batch2       P013        T0      A
10       S20   control batch3       P017        T1      A
5        S21   control batch2       P008        T0      A
9        S03   treated batch2       P017        T1      A
3        S04   treated batch3       P044        T0      A
8        S05   treated batch3       P020        T1      A

Task2_dataframe: 0.017 sec elapsed
Media dei conteggi Top 100 Geni (prime 10 righe):
   cohort condition   gene_id mean_count
1       A   control GENE_0006   34.00000
2       B   control GENE_0006   35.83333
3       A   treated GENE_0006   42.77778
4       B   treated GENE_0006   44.33333
5       A   control GENE_0020   30.66667
6       B   control GENE

In [7]:
#tabella confronto tempi
T_DT1 <- 0.029
T_DT2 <- 0.052
T_DT3 <- 0.069
T_DF1 <- 0.020
T_DF2 <- 0.017
T_DF3 <- 0.066

# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "Task 2",
           "Task 3"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT2, T_DT3),
  Tempo_data.frame_Sec = c(T_DF1, T_DF2, T_DF3)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

    Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1 Task 1                0.029                0.020              0.7
2 Task 2                0.052                0.017              0.3
3 Task 3                0.069                0.066              1.0
